In [2]:
import pandas as pd
import numpy as np
import requests
import os 
from dotenv import load_dotenv
import requests
from tqdm import tqdm
import pickle
import re
pd.options.display.max_columns = None
from fuzzywuzzy import process, fuzz
from collections import Counter

import sys
sys.path.append("../")
import src.soporteAPIs as sa
sys.path.append("../")
import src.biblioteca as bb

# Importing pickle file

In [3]:
with open('../data/pickle/artist.pickle', 'rb') as tracks:
    tracks = pickle.load(tracks)

In [4]:
tracks.head()

,url,artist_clean,track_clean
0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert)
1,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love
2,https://open.spotify.com/track/5aAx2yezTd8zXrk...,"the weeknd, daft punk",starboy
3,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"the chainsmokers, halsey",closer
4,https://open.spotify.com/track/6fujklziTHa8uoM...,"rae sremmurd, gucci mane",black beatles


In [5]:
tracks.rename(columns={'track_clean':'track','artist_clean':'artist'}, inplace=True)
tracks.head()

,url,artist,track
0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert)
1,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love
2,https://open.spotify.com/track/5aAx2yezTd8zXrk...,"the weeknd, daft punk",starboy
3,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"the chainsmokers, halsey",closer
4,https://open.spotify.com/track/6fujklziTHa8uoM...,"rae sremmurd, gucci mane",black beatles


I will split the artist column in order to split between main artist (`artist_0`) and possible featuring artist (`artist_1`, `artist_1`)

In [6]:
test = tracks['artist'].str.split(',', n=2, expand = True)
test

,0,1,2
0,migos,None,None
1,drake,None,None
2,the weeknd,daft punk,None
3,the chainsmokers,halsey,None
4,rae sremmurd,gucci mane,None
...,...,...,...
454434,fase yoda,None,None
454442,max,ali gatie,None
454443,cooper alan,rvshvd,None
454451,kali,None,None


In [7]:
tracks = pd.concat([tracks, test], axis = 1)
tracks.head()

,url,artist,track,0,1,2
0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),migos,None,None
1,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love,drake,None,None
2,https://open.spotify.com/track/5aAx2yezTd8zXrk...,"the weeknd, daft punk",starboy,the weeknd,daft punk,None
3,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"the chainsmokers, halsey",closer,the chainsmokers,halsey,None
4,https://open.spotify.com/track/6fujklziTHa8uoM...,"rae sremmurd, gucci mane",black beatles,rae sremmurd,gucci mane,None


In [8]:
tracks.rename(columns={0:'artist_0', 1:'artist_1', 2:'artist_2'}, inplace=True)
tracks.head()

,url,artist,track,artist_0,artist_1,artist_2
0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),migos,None,None
1,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love,drake,None,None
2,https://open.spotify.com/track/5aAx2yezTd8zXrk...,"the weeknd, daft punk",starboy,the weeknd,daft punk,None
3,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"the chainsmokers, halsey",closer,the chainsmokers,halsey,None
4,https://open.spotify.com/track/6fujklziTHa8uoM...,"rae sremmurd, gucci mane",black beatles,rae sremmurd,gucci mane,None


# lastfm
In this part of the process I will extract form the LastFM API all the data related to the tracks and artist genres.

First, I need to configure my credentials.

In [9]:
load_dotenv()

True

In [10]:
api = os.getenv("lastfm-id")
username = os.getenv("lastfm-user")

Testing to gather the data from the api with a function:

In [11]:
res = sa.getLastFMData({
    'method': 'artist.getInfo',
    'artist':'Harry Styles',
})
res.json()['artist']['bio']

{'links': {'link': {'#text': '',
   'rel': 'original',
   'href': 'https://last.fm/music/Harry+Styles/+wiki'}},
 'published': '24 Aug 2011, 17:11',
 'summary': 'Harry Edward Styles (born 1 February 1994 in Redditch, Worcestershire, England) is a Grammy-winning, British singer, songwriter, and actor. As a member of the British (&Irish) boy band One Direction, singer Harry Styles topped the charts, toured the world, and sold millions of albums before going solo in 2016. Styles was raised in Holmes Chapel, Cheshire, and he made his first foray into music with a high-school band named White Eskimo. In 2010, he left home and joined the seventh season of The X Factor <a href="https://www.last.fm/music/Harry+Styles">Read more on Last.fm</a>',
 'content': 'Harry Edward Styles (born 1 February 1994 in Redditch, Worcestershire, England) is a Grammy-winning, British singer, songwriter, and actor. As a member of the British (&Irish) boy band One Direction, singer Harry Styles topped the charts, to

First I try on a slice of my dataframe

In [12]:
tqdm.pandas()

test = tracks.sample(100)
test[['bio', 'artist_tag']] = test.progress_apply(lambda x: sa.getArtistInfo(x.artist_0), axis=1, result_type='expand')

100%|██████████| 100/100 [00:17<00:00,  5.61it/s]


In [13]:
test.head()

,url,artist,track,artist_0,artist_1,artist_2,bio,artist_tag
437743,https://open.spotify.com/track/3EgYiwkoFHHn1MS...,dreamspook,fear in love,dreamspook,None,None,"{'links': {'link': {'#text': '', 'rel': 'origi...",[]
372667,https://open.spotify.com/track/1MIGkQxcdAt2lDx...,"jack harlow, pooh shiesty",suvs (black on black),jack harlow,pooh shiesty,None,"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'rap', 'url': 'https://www.last.fm/t..."
346660,https://open.spotify.com/track/7kjg2NCn3Zx70m1...,gil scott-heron,the revolution will not be televised,gil scott-heron,None,None,"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'soul', 'url': 'https://www.last.fm/..."
87019,https://open.spotify.com/track/0lnIJmgcUpEpe4A...,beyoncé,sorry,beyoncé,None,None,"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'rnb', 'url': 'https://www.last.fm/t..."
373026,https://open.spotify.com/track/5PLkBydBHc2g4uY...,"steven malcolm, andy mineo, hollyn",party in the hills,steven malcolm,andy mineo,hollyn,"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'Hip-Hop', 'url': 'https://www.last...."


This is for the whole dataframe:

In [14]:
tqdm.pandas()

#test = tracks.sample(100)
tracks[['bio', 'artist_tag']] = tracks.progress_apply(lambda x: sa.getArtistInfo(x.artist_0), axis=1, result_type='expand')

100%|██████████| 14659/14659 [42:15<00:00,  5.78it/s] 


In [15]:
tracks.head()

,url,artist,track,artist_0,artist_1,artist_2,bio,artist_tag
0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),migos,None,None,"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'Hip-Hop', 'url': 'https://www.last...."
1,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love,drake,None,None,"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'Hip-Hop', 'url': 'https://www.last...."
2,https://open.spotify.com/track/5aAx2yezTd8zXrk...,"the weeknd, daft punk",starboy,the weeknd,daft punk,None,"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'rnb', 'url': 'https://www.last.fm/t..."
3,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"the chainsmokers, halsey",closer,the chainsmokers,halsey,None,"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'electronic', 'url': 'https://www.la..."
4,https://open.spotify.com/track/6fujklziTHa8uoM...,"rae sremmurd, gucci mane",black beatles,rae sremmurd,gucci mane,None,"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'Hip-Hop', 'url': 'https://www.last...."


Now I get the tags for each individual track:

In [16]:
res = sa.getLastFMData({
    'method': 'track.getInfo',
    'artist':'Harry Styles',
    'track':'As It Was'
})
res.json()['track']['toptags']['tag']#['published']#[0]['name']

[{'name': 'pop', 'url': 'https://www.last.fm/tag/pop'},
 {'name': 'rock', 'url': 'https://www.last.fm/tag/rock'},
 {'name': 'indie pop', 'url': 'https://www.last.fm/tag/indie+pop'},
 {'name': 'synthpop', 'url': 'https://www.last.fm/tag/synthpop'},
 {'name': 'british', 'url': 'https://www.last.fm/tag/british'}]

This is how I will get info for a slice of my dataframe:

In [17]:
tqdm.pandas()

test = tracks.sample(100)
test[['track_tag', 'published_track']] = test.progress_apply(lambda x: sa.getTrackTags(x.artist_0, x.track), axis=1, result_type='expand')

100%|██████████| 100/100 [00:19<00:00,  5.09it/s]


In [18]:
print(test['track_tag'].isnull().sum())
test.head()

59


,url,artist,track,artist_0,artist_1,artist_2,bio,artist_tag,track_tag,published_track
381027,https://open.spotify.com/track/3jjsRKEsF42ccXf...,mitski,washing machine heart,mitski,None,None,"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'seen live', 'url': 'https://www.las...","[{'name': 'art rock', 'url': 'https://www.last...","14 Jan 2022, 03:05"
323849,https://open.spotify.com/track/5Tutv8aK0ZJFcil...,brockhampton,chain on (feat. jpegmafia),brockhampton,None,None,"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'Hip-Hop', 'url': 'https://www.last....","[{'name': 'hip hop', 'url': 'https://www.last....","09 Nov 2022, 03:29"
398913,https://open.spotify.com/track/0tBbt8CrmxbjRP0...,"selena gomez, marshmello",wolves,selena gomez,marshmello,None,"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'pop', 'url': 'https://www.last.fm/t...","[{'name': 'pop', 'url': 'https://www.last.fm/t...","29 Oct 2017, 02:07"
263287,https://open.spotify.com/track/0Tcw8w9pvPrZoOR...,"future, lil uzi vert",over your head,future,lil uzi vert,None,"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'Hip-Hop', 'url': 'https://www.last....",NaN,NaN
153415,https://open.spotify.com/track/3xB5uHrhe0uaIxN...,g-eazy,i wanna rock (feat. gunna),g-eazy,None,None,"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'Hip-Hop', 'url': 'https://www.last....","[{'name': 'rap', 'url': 'https://www.last.fm/t...","31 Jan 2022, 18:03"


In [19]:
test.tail()

,url,artist,track,artist_0,artist_1,artist_2,bio,artist_tag,track_tag,published_track
1456,https://open.spotify.com/track/1tyjWMMINb9S0YB...,london on da track,"no flag (feat. nicki minaj, 21 savage & offset)",london on da track,None,None,"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'Hip-Hop', 'url': 'https://www.last....",NaN,NaN
281638,https://open.spotify.com/track/3gwoz4xZuye0agj...,troye sivan,easy,troye sivan,None,None,"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'pop', 'url': 'https://www.last.fm/t...","[{'name': 'synthpop', 'url': 'https://www.last...","24 Jan 2022, 00:22"
424340,https://open.spotify.com/track/1BkSadCalO2qea7...,"big boi, gucci mane, pimp c",in the south,big boi,gucci mane,pimp c,"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'Hip-Hop', 'url': 'https://www.last....",NaN,NaN
240772,https://open.spotify.com/track/7knsEeZDIqMFA3e...,oli fox,cruel,oli fox,None,None,"{'links': {'link': {'#text': '', 'rel': 'origi...",[],NaN,NaN
349972,https://open.spotify.com/track/2Jy50uaAgvyyrVO...,levi hummon,wedding dress,levi hummon,None,None,"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'country', 'url': 'https://www.last....",NaN,NaN


This is for the whole dataset:

In [25]:
tqdm.pandas()

tracks[['track_tag', 'published_track']] = tracks.progress_apply(lambda x: sa.getTrackTags(x.artist_0, x.track), axis=1, result_type='expand')

 44%|████▍     | 6453/14659 [20:44<27:30,  4.97it/s]  

In [22]:
print(tracks['track_tag'].isnull().sum(), tracks['artist_tag'].isnull().sum())
tracks.head()

9301 1


,url,artist,track,artist_0,artist_1,artist_2,bio,artist_tag,track_tag,published_track
0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),migos,None,None,"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'Hip-Hop', 'url': 'https://www.last....","[{'name': 'trap', 'url': 'https://www.last.fm/...","30 Jan 2018, 04:41"
1,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love,drake,None,None,"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'Hip-Hop', 'url': 'https://www.last....","[{'name': 'trap', 'url': 'https://www.last.fm/...","27 Oct 2016, 02:34"
2,https://open.spotify.com/track/5aAx2yezTd8zXrk...,"the weeknd, daft punk",starboy,the weeknd,daft punk,None,"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'rnb', 'url': 'https://www.last.fm/t...","[{'name': 'rnb', 'url': 'https://www.last.fm/t...","23 Sep 2016, 01:07"
3,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"the chainsmokers, halsey",closer,the chainsmokers,halsey,None,"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'electronic', 'url': 'https://www.la...","[{'name': '2016', 'url': 'https://www.last.fm/...","31 Jul 2016, 22:59"
4,https://open.spotify.com/track/6fujklziTHa8uoM...,"rae sremmurd, gucci mane",black beatles,rae sremmurd,gucci mane,None,"{'links': {'link': {'#text': '', 'rel': 'origi...","[{'name': 'Hip-Hop', 'url': 'https://www.last....","[{'name': 'trap', 'url': 'https://www.last.fm/...","27 Oct 2016, 05:23"


# Data Cleaning

Cleaning the new `bio` data

In [ ]:
bio = tracks['bio'].apply(pd.Series)
bio.head()
#artist_tag['tag'] = artist_tag['tag'].apply(pd.Series)

Cleaning the `artist_tag` column

In [ ]:
artist_tag = tracks['artist_tag'].apply(pd.Series)
new_columns = []
for item in artist_tag.columns.to_list():
    new_columns.append('aritist_genre_'+str(item))
artist_tag.columns = new_columns
#print(artist_tag.columns.to_list(), new_columns)
artist_tag.head()

In [ ]:
# cleaning unnecesary dictionaries
for index, row in artist_tag.iterrows():
    for column in artist_tag.columns.tolist():
        try:
            row[column] = list(row[column].values())[0] 
            #print(row[column])
        except:
            continue
artist_tag.head()

Cleaning `track_tag` column:

In [ ]:
track_tag = tracks['track_tag'].apply(pd.Series)
new_columns = []
for item in track_tag.columns.to_list():
    new_columns.append('track_genre_'+str(item))
track_tag.columns = new_columns
track_tag.head()

In [ ]:
# cleaning unnecesary dictionaries
for index, row in track_tag.iterrows():
    for column in track_tag.columns.tolist():
        try:
            row[column] = list(row[column].values())[0] 
        except:
            continue
track_tag.head()

Merging the clean columns to the original dataframe and removing unnecesary columns:

In [ ]:
tracks_final = pd.concat([tracks, bio, artist_tag, track_tag], axis = 1)
tracks_final.head()

In [ ]:
tracks_final.drop(['bio', 'artist_tag', 'track_tag', 'links', 0], axis=1, inplace=True)
tracks_final.head()

# Adding new columns

Generating a new clean genres column based on the extracted ones from de apis.

In [ ]:
tracks_final.head(1)

In [ ]:
tracks_final['music_genre'] = np.where(tracks_final['aritist_genre_0'].isnull() == True, tracks_final['track_genre_0'].str.lower(), tracks_final['aritist_genre_0'].str.lower())
tracks_final['music_genre'].value_counts()

In [ ]:
cuenta_generos = Counter(genres for genres in tracks_final['music_genre'])
print(f"Hay {len(cuenta_generos)} generos diferentes.")

In [ ]:
dict_genres = dict(cuenta_generos.most_common(25))

In [ ]:
fuzz.ratio('hip-hop', "hip hop")

In [ ]:
tracks_final["clean_music_genre"] = tracks_final.apply(lambda x: sa.music_genres(x["music_genre"], dict_genres), axis = 1)
tracks_final["clean_music_genre"].value_counts()

Adding gender info column.

In [ ]:
tracks_final.sample()

In [ ]:
tracks_final.head(1)

In [ ]:
#df["gender"] = df.apply(lambda x: generos(x['content'], bb.dict_gender), axis=1)
tracks_final["gender"] = tracks_final['summary'].apply(sa.generos)

In [ ]:
tracks_final['gender'].value_counts() / tracks_final.shape[0]

# EDA

Total number of rows and columns of the dataframe

In [ ]:
tracks_final.shape

Counting duplicated rows

In [ ]:
tracks_final.duplicated().sum()

Counting null values

In [ ]:
tracks_final.isnull().sum() / tracks_final.shape[0]

In [ ]:
tracks_final.info()

The only null columns available on the dataset are on the `streams` column. This one I will enrich it with api info.

Main statistic values for numeric columns

In [ ]:
tracks_final.describe().T

Main statistic values for categorical columns

In [ ]:
tracks_final.describe(include='object').T

# Export

In [ ]:
with open('../data/pickle/lastfm.pickle', 'wb') as data_lastfm:
    pickle.dump(tracks_final, data_lastfm)